# Analysis 11: Calculate Region HDR Overlaps

Based on `DivergentRegion_Overlaps.R` script from Analysis GitHub repo.

Take the HVR regions present in the mapping panel and measure the overlap of intervals detected for each toxicant.

In [ ]:
library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:lubridate':

    intersect, setdiff, union

The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min

Loading required package: S4Vectors

Attaching package: 'S4Vectors'

The following objects are masked from 'package:lubridate':

    second, second<-

The following


Attaching package: 'data.table'

The following object is masked from 'package:GenomicRanges':

    shift

The following object is masked from 'package:IRanges':

    shift

The following objects are masked from 'package:S4Vectors':

    first, second

The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from 'package:dplyr':

    between, first, last

The following object is masked from 'package:purrr':

    transpose

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: 'plyr'

The following object is masked from 'package:IRanges':

    desc

The following object is masked from 'package:S4Vectors':

    rename

The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following object is masked from 'package:purrr':

    compact

# Inputs

-   `hd_regions_file`: path to the `hvr_gwas_strains.tsv` file generated by `analysis/hdr_summarize_strain.qmd`
-   `ns_folder`: path to the nemascan analysis folder

In [ ]:

# path to HDR regions file for GWAS strains from analysis/hdr_summarize_strain.qmd
hd_regions_file <- "data/processed/HDR_overlap/hvr_gwas_strains.tsv"

# Path to folder with NemaScan results
ns_folder <- "data/processed/20231116_Analysis_NemaScan"


# Outputs

In the output directory (`data/processed/HVR_Overlap/All_QTL_HD_overlap`) there is a file generated for each trait (e.g `length_2_4_D_HD_region_overlap.tsv`). Each row in the file represented a qtl detected for that trait. These files are used later in the `triage` scripts.

In [ ]:
outdir <- glue::glue("data/processed/HVR_Overlap/All_QTL_HD_overlap")

#check if the outdir exists, if not create it
if (!dir.exists(outdir)) {
    dir.create(outdir)
}


Warning in dir.create(outdir): cannot create dir
'data/processed/HVR_Overlap/All_QTL_HD_overlap', reason 'No such file or
directory'

Warning in dir.create(inbred_out_dir): cannot create dir
'data/processed/HVR_Overlap/All_QTL_HD_overlap/inbred', reason 'No such file or
directory'

# Functions

In [ ]:

create_proc_mapping <- function(mapping_file) {
    mapping <- fread(mapping_file)
    mapping <- mapping %>%
        filter(!is.na(peak_id))
    
    n_peak_markers <- length(unique(mapping$peak_id))
    print("Number of markers with a peak ID")
    print(n_peak_markers)
    
    if (n_peak_markers == 0) {
        mapping_list <- NULL
    }
    mapping_list <- list()
    for (i in 1:n_peak_markers) {
        
        
        fil_mapping <- mapping %>%
            filter(peak_id == i)
        
        peak_marker <- unique(fil_mapping$marker)

        ref_strains <- fil_mapping %>%
            filter(allele == -1) %>%
            distinct(strain) %>%
            pull(strain)
        
        alt_strains <- fil_mapping %>%
            filter(allele == 1) %>%
            distinct(strain) %>%
            pull(strain)
        
        single_peak_df <- fil_mapping  %>% 
            select(CHROM, startPOS, endPOS) %>%
            distinct()
        
        #Create a genomic ranges object for the peak
        peak_gr <- GRanges(seqnames = single_peak_df$CHROM,
                           ranges = IRanges(start = single_peak_df$startPOS,
                                            end = single_peak_df$endPOS),

        )

        peak_data <- list(ref_strains = ref_strains, alt_strains = alt_strains, peak_gr = peak_gr)

        mapping_list[[peak_marker]] <- peak_data
    }
    
    return(mapping_list)
}

# Function to create a genomic ranges object for the HD regions for the strain set

create_hd_gr <- function(hd_file) {
    hd_df <- data.table::fread(
        hd_file,
        col.names = c("chrom", "start", "stop", "hd_region", "strains", "n_strains")
    )
    hd_gr <- GRanges(seqnames = hd_df$chrom,
                     ranges = IRanges(start = hd_df$start,
                                      end = hd_df$end),
                     hd_region = hd_df$hd_region,
                     strains = hd_df$strains,
                     n_strains = hd_df$n_strains
                     )
    return(hd_gr)
}

#Create a function to check if a peak overlaps with a HD region
strains_with_hd_overlap <- function(peak_gr, hd_gr) {
    #return only the ranges in the hd_gr that overlap with the peak_gr
    hd_overlap <- subsetByOverlaps(hd_gr, peak_gr)
    
    n_hd_overlaps <- length(hd_overlap)
    print( "Number of HD regions that overlap with a QTL region")
    print(n_hd_overlaps)

    #return the hd_overlap object as a data frame
    hd_overlap_df <- as.data.frame(hd_overlap) %>% 
    #split the strains column into a list
        separate_rows(strains, sep = ",") 

    hd_overlap_strains <- hd_overlap_df %>%
        select(strains) %>%
        distinct() %>% 
        pull(strains)
    
    return(hd_overlap_strains)
}

#Load in the mapping data
#example_mapping_file <- "/projects/b1059/projects/Tim/2021_GWA_manuscript/data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Carboxin_AGGREGATE_mapping_inbred.tsv" 
#mapping_list <- create_proc_mapping(example_mapping_file)
#head(mapping_list)

#Load in the HD regions data
#hd_regions_file <- "data/20231121_divergent_regions_strain.bed"
#hd_gr <- create_hd_gr(hd_regions_file)
#head(hd_gr)

create_trait_overlaps <- function(mapping_file, hd_regions_file, outdir){

#Create mapping files obj
mapping_list <- create_proc_mapping(mapping_file)
#if mapping list is null, return null
if (is.null(mapping_list)) {
    stop("No markers with peak IDs")
}

#Create HD regions obj
hd_gr <- create_hd_gr(hd_regions_file)

#Iterate through the mapping list and check for HD region overlaps
hd_overlap_list <- list()

for (i in 1:length(mapping_list)) {
    peak_data <- mapping_list[[i]]
    #get the marker id for the peak from the name of the list element
    peak_marker <- names(mapping_list[i])
    print(peak_marker)
    peak_gr <- peak_data$peak_gr
    
    #calculate the number of strains with either ref or alt status at the peak marker
    n_strains_ref <- length(peak_data$ref_strains)
    n_strains_alt <- length(peak_data$alt_strains)

    total_strains <- n_strains_ref + n_strains_alt
    
    hd_overlap_strains <- strains_with_hd_overlap(peak_gr, hd_gr)
    
    n_strains_ref_overlap <- length(intersect(peak_data$ref_strains, hd_overlap_strains))
    n_strains_alt_overlap <- length(intersect(peak_data$alt_strains, hd_overlap_strains))

    hd_overlap_list[[peak_marker]] <- list(
                                            peak_marker = peak_marker,
                                            n_strains_ref = n_strains_ref,
                                            n_strains_alt = n_strains_alt,
                                            total_strains = total_strains,
                                            n_strains_ref_overlap = n_strains_ref_overlap,
                                            n_strains_alt_overlap = n_strains_alt_overlap,
                                            prop_ref_overlap = n_strains_ref_overlap/n_strains_ref,
                                            prop_alt_overlap = n_strains_alt_overlap/n_strains_alt
                                           )
}

head(hd_overlap_list)

hd_overlap_df <- ldply(hd_overlap_list, data.frame)

#extract the trait name from the mapping file name
get_traitname <- function(fn){
    bn <- basename(fn)
    if(str_detect(bn, "CV_length")){
        #extract everthing before _AGGREGATE
        traitname <- str_extract(bn, ".*(?=_AGGREGATE)")
        traitname <- str_remove(traitname, "processed_")
    }else{
        #extract everything before _AGGREGATE 
        traitname <- str_extract(bn, ".*(?=_AGGREGATE)")
        traitname <- str_remove(traitname, "processed_")
    }
    return(traitname)
}
trait_name <- get_traitname(mapping_file)

print(trait_name)

hd_overlap_df$trait_name <- trait_name

#Add the algoirthm to the data frame
get_algorithm <- function(fn){
    bn <- basename(fn)
    if(str_detect(bn, "inbred")){
        algorithm <- "inbred"
    }else{
        algorithm <- "loco"
    }
    return(algorithm)
}
algorithm <- get_algorithm(mapping_file)
hd_overlap_df$algorithm <- algorithm


head(hd_overlap_df)

#write out the data frame with the HD region overlap data using the trait name
#hd_overlap_outfile <- paste0("data/", trait_name, "_HD_region_overlap.tsv")
hd_overlap_outfile <- glue::glue("{outdir}/{trait_name}_HD_region_overlap.tsv")

# Check if the file exists
if (file.exists(hd_overlap_outfile)) {
    # If the file exists, read the existing data
    existing_data <- read.table(hd_overlap_outfile, header = TRUE, sep = "\t")
    
    # Append the new data to the existing data
    combined_data <- rbind(existing_data, hd_overlap_df)
} else {
    # If the file does not exist, the new data is the combined data
    combined_data <- hd_overlap_df
}

# Write the combined data to the file
write.table(combined_data, hd_overlap_outfile, sep = "\t", row.names = FALSE)

}


# Main

The main function executed by this script is `create_trait_overlaps()` which executes a series of other functions defined in the script.

1.  Create a mapping files object (a list with several named elements) with `create_proc_mapping()`
    -   Function loads the mapping file containing the p-values for all markers and filters to just the peak marker for each interval that was defined by NS.
    -   Using the filtered mapping data the function counts a list of strains with the REF or ALT allele peak marker allele. Generates the `ref_strains` and `alt_strains` elements of the mapping files object.
    -   Creates a genomic ranges object using data about the interval that peak marker represents. This is the `peak_gr` element of the mapping files object.
2.  Create a genomic range object from the HVR bedfile using the `create_hd_gr()` function.
3.  Identify strains with that have an HVR in the region of the peak marker using the `strains_with_hd_overlap()` function.
4.  Calculate the proportion of strains with each peak marker allele REF or ALT that have an HVR overlap.

In [ ]:
#Get the mapping directories
inbred_mapping_dir <- glue::glue("{ns_folder}/INBRED/Mapping/Processed")

#Get the mapping files
inbred_mapping_files <- list.files(
    path = inbred_mapping_dir,
    pattern = "AGGREGATE_mapping_inbred",
    recursive = T, full.names = T)

#Get rid of CV length files
inbred_mapping_files <- inbred_mapping_files[!grepl("_CV_", inbred_mapping_files)]


#Iterate through the mapping files and create the HD region overlap data
for (i in 1:length(inbred_mapping_files)) {
    mapping_file <- inbred_mapping_files[i]
    print(mapping_file)
    tryCatch(create_trait_overlaps(mapping_file, hd_regions_file, inbred_out_dir),
                error = function(e) print(e)
                )
    #create_trait_overlaps(mapping_file, hd_regions_file, outdir)
}


[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_2_4_D_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 1
[1] "V:14453636"
[1] "Number of HD regions that overlap with a QTL region"
[1] 10
[1] "length_2_4_D"

Warning in file(file, ifelse(append, "a", "w")): cannot open file
'data/processed/HVR_Overlap/All_QTL_HD_overlap/inbred/length_2_4_D_HD_region_overlap.tsv':
No such file or directory

<simpleError in file(file, ifelse(append, "a", "w")): cannot open the connection>
[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Aldicarb_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 5
[1] "III:8311919"
[1] "Number of HD regions that overlap with a QTL region"
[1] 0
[1] "IV:2368192"
[1] "Number of HD regions that overlap with a QTL region"
[1] 6
[1] "IV:13208614"
[1] "Number of HD regions that overlap with a QTL region"
[1] 2
[1] "IV:16631628"
[1] "Number of HD regions that overlap with a QTL region"
[1] 29
[1] "X:13252751"
[1] "Number of HD regions that overlap with a QTL region"
[1] 8
[1] "length_Aldicarb"

Warning in file(file, ifelse(append, "a", "w")): cannot open file
'data/processed/HVR_Overlap/All_QTL_HD_overlap/inbred/length_Aldicarb_HD_region_overlap.tsv':
No such file or directory

<simpleError in file(file, ifelse(append, "a", "w")): cannot open the connection>
[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Arsenic_trioxide_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 2
[1] "II:13692527"
[1] "Number of HD regions that overlap with a QTL region"
[1] 76
[1] "III:1642838"
[1] "Number of HD regions that overlap with a QTL region"
[1] 198
[1] "length_Arsenic_trioxide"

...

<simpleError in file(file, ifelse(append, "a", "w")): cannot open the connection>
[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Triphenyl_phosphate_50_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 0
<simpleError in mapping_list[[peak_marker]] <- peak_data: attempt to select less than one element in OneIndex>
[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Triphenyl_phosphate_6_25_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 2
[1] "IV:16034033"
[1] "Number of HD regions that overlap with a QTL region"
[1] 5
[1] "V:7623783"
[1] "Number of HD regions that overlap with a QTL region"
[1] 229
[1] "length_Triphenyl_phosphate_6_25"

Warning in file(file, ifelse(append, "a", "w")): cannot open file
'data/processed/HVR_Overlap/All_QTL_HD_overlap/inbred/length_Triphenyl_phosphate_6_25_HD_region_overlap.tsv':
No such file or directory

<simpleError in file(file, ifelse(append, "a", "w")): cannot open the connection>
[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Water_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 0
<simpleError in mapping_list[[peak_marker]] <- peak_data: attempt to select less than one element in OneIndex>
[1] "data/processed/20231116_Analysis_NemaScan/INBRED/Mapping/Processed/processed_length_Zinc_dichloride_AGGREGATE_mapping_inbred.tsv"
[1] "Number of markers with a peak ID"
[1] 1
[1] "I:4884561"
[1] "Number of HD regions that overlap with a QTL region"
[1] 2
[1] "length_Zinc_dichloride"

Warning in file(file, ifelse(append, "a", "w")): cannot open file
'data/processed/HVR_Overlap/All_QTL_HD_overlap/inbred/length_Zinc_dichloride_HD_region_overlap.tsv':
No such file or directory

<simpleError in file(file, ifelse(append, "a", "w")): cannot open the connection>